<a href="https://colab.research.google.com/github/gihantha-sanjana/CMP7005-Assingment-Work-Repo/blob/main/Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# **Load preprocessed data with DATETIME as index**

In [3]:
data = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/DatasetCMP7005/MergeDataSet/Preprocessed-Data.csv", index_col=0, parse_dates=True)
print(data.head())

                     PM2.5  PM10   SO2   NO2     CO    O3  TEMP    PRES  \
DATETIME                                                                  
2013-03-01 00:00:00    4.0   4.0   4.0   7.0  300.0  77.0  -0.7  1023.0   
2013-03-01 01:00:00    8.0   8.0   4.0   7.0  300.0  77.0  -1.1  1023.2   
2013-03-01 02:00:00    7.0   7.0   5.0  10.0  300.0  73.0  -1.1  1023.5   
2013-03-01 03:00:00    6.0   6.0  11.0  11.0  300.0  72.0  -1.4  1024.5   
2013-03-01 04:00:00    3.0   3.0  12.0  12.0  300.0  72.0  -2.0  1025.2   

                     DEW_POINT  RAIN     WD    WS       STATION   HUMIDITY  \
DATETIME                                                                     
2013-03-01 00:00:00      -18.8   0.0  337.5  4.15  Aotizhongxin  23.878120   
2013-03-01 01:00:00      -18.2   0.0    0.0  4.15  Aotizhongxin  25.880955   
2013-03-01 02:00:00      -18.2   0.0  337.5  4.15  Aotizhongxin  25.880955   
2013-03-01 03:00:00      -19.4   0.0  315.0  3.10  Aotizhongxin  23.872562   
2013-0

In [4]:
data.dtypes

,0
PM2.5,float64
PM10,float64
SO2,float64
NO2,float64
CO,float64
O3,float64
TEMP,float64
PRES,float64
DEW_POINT,float64
RAIN,float64


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Load and preprocess the data
# Assuming `data` is already loaded and has DATETIME as the index
# Ensure index is datetime
data.index = pd.to_datetime(data.index, errors='coerce')

# Create lag features
lags = 3  # Number of lagged time steps
for i in range(1, lags + 1):
    data[f"lag_{i}"] = data['MAX_AQI'].shift(i)

# Drop missing values introduced by lagging
data = data.dropna()

# Separate features and targets
X = data.drop(['MAX_AQI', 'AQI_CATEGORY'], axis=1)  # Features
y_reg = data['MAX_AQI']  # Target for regression
y_clf = data['AQI_CATEGORY']  # Target for classification

# Encode the classification target
label_encoder = LabelEncoder()
y_clf = label_encoder.fit_transform(y_clf)

# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Define preprocessing for numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),            # Scale numeric columns
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # Encode categorical columns
    ]
)

# Preprocess the features
X_processed = preprocessor.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train_reg, y_test_reg = train_test_split(X_processed, y_reg, test_size=0.2, random_state=42, shuffle=False)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_processed, y_clf, test_size=0.2, random_state=42, shuffle=False)

# --------------------
# Train Models

# Results storage
results = {}

# Regression Models
# 1. Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train_reg)
y_pred_reg = lin_reg.predict(X_test)
results['Linear Regression'] = {
    'MAE': mean_absolute_error(y_test_reg, y_pred_reg),
    'R2': r2_score(y_test_reg, y_pred_reg)
}

# 2. Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train, y_train_reg)
y_pred_reg = rf_reg.predict(X_test)
results['Random Forest Regressor'] = {
    'MAE': mean_absolute_error(y_test_reg, y_pred_reg),
    'R2': r2_score(y_test_reg, y_pred_reg)
}

# 3. XGBoost Regressor
xgb_reg = xgb.XGBRegressor(random_state=42)
xgb_reg.fit(X_train, y_train_reg)
y_pred_reg = xgb_reg.predict(X_test)
results['XGBoost Regressor'] = {
    'MAE': mean_absolute_error(y_test_reg, y_pred_reg),
    'R2': r2_score(y_test_reg, y_pred_reg)
}

# --------------------
# Display Results
import pprint
pprint.pprint(results)


{'Linear Regression': {'MAE': 16.461926235541743, 'R2': 0.9131045776760967},
 'Random Forest Regressor': {'MAE': 1.9961190287031485,
                             'R2': 0.9815714735415095},
 'XGBoost Regressor': {'MAE': 3.560148699550873, 'R2': 0.9802915955990372}}


# **After Looking Above values i decide to select Random Forest Regression because it Performance are excellent, Low Errors and R2 value is high.**

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

# Load the dataset
data = pd.read_csv(
    "/content/drive/MyDrive/ColabNotebooks/DatasetCMP7005/MergeDataSet/Preprocessed-Data.csv",
    index_col=0,
    parse_dates=True
)

# Ensure index is datetime
data.index = pd.to_datetime(data.index, errors='coerce')

# Create lag features
lags = 3  # Number of lagged time steps
for i in range(1, lags + 1):
    data[f"lag_{i}"] = data['MAX_AQI'].shift(i)

# Drop missing values introduced by lagging
data = data.dropna()

# Features and target
X = data.drop(['MAX_AQI', 'AQI_CATEGORY'], axis=1)  # Exclude target variable
y = data['MAX_AQI']  # Target variable

# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Preprocess categorical and numeric data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),            # Scale numeric columns
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # Encode categorical columns
    ]
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Train Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train_processed, y_train)

# Evaluate the model
y_pred = rf_reg.predict(X_test_processed)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest Regressor Performance:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R2 Score: {r2}")

# Save the trained model
joblib.dump(rf_reg, "random_forest_model.pkl")
print("Model saved as random_forest_model.pkl")

# Save the preprocessor for deployment
joblib.dump(preprocessor, "preprocessor.pkl")
print("Preprocessor saved as preprocessor.pkl")


Random Forest Regressor Performance:
Mean Absolute Error (MAE): 1.992728032757009
R2 Score: 0.9814810716858263
Model saved as random_forest_model.pkl
Preprocessor saved as preprocessor.pkl
